In [ ]:
# !pip install psycopg2-binary
# import psycopg2

In [ ]:
import psycopg2
db_config = {
#    'host': '192.168.0.21',
    'host': '192.168.0.21',
    'database': 'wms_legacy',
    'user': 'postgres',
    'password': 'iTc3D3A24#24',
    'port': '5432'  # Port default PostgreSQL adalah 5432
}

# Membuat koneksi ke database
try:
    connection = psycopg2.connect(**db_config)
    cursor = connection.cursor()
    print("Berhasil terhubung ke database PostgreSQL")

    # Lakukan operasi database di sini...

except (Exception, psycopg2.Error) as error:
    print("Error saat terhubung ke database PostgreSQL:", error)

Berhasil terhubung ke database PostgreSQL


In [ ]:
import re
import pandas as pd

file_names = ['STOCKM_C011_20240123.xlsx', 'STOCKM_C011_20240124.xlsx']

def extract_date_from_filename(file_name):
    match = re.search(r'_(\d{8})\.xlsx', file_name)
    if match:
        date_str = match.group(1)
        return pd.to_datetime(date_str, format='%Y%m%d')
    else:
        return None

for file_name in file_names:
    date = extract_date_from_filename(file_name)
    print(f"File: {file_name}, Date: {date}")


File: STOCKM_C011_20240123.xlsx, Date: 2024-01-23 00:00:00
File: STOCKM_C011_20240124.xlsx, Date: 2024-01-24 00:00:00


In [ ]:
# conect to ftp and postgres
# specific folder and file in you want
# Read Excel file directly from FTP and check the field from data
# upload data to postgres
# extract date form soh file
# query distinct tgl_import date from locaton history
# find mising date
# run script to append new date data
# create view KA today where tgl_import date = today
# determine comment columns date ka soh and wms location

Connect to ftp and postgres

In [7]:
from ftplib import FTP
import psycopg2
import pandas as pd
import re, os
from io import BytesIO
from zipfile import ZipFile
from sqlalchemy import create_engine
import math
# from openpyxl import load_workbook

# Replace these values with your FTP server details
ftp_server ='order.kacoldstorage.com'
username = 'c011'
password = '110c'

# pg_host = 'localhost'
# pg_database = 'Belajar'
# pg_user = 'postgres'
# pg_password = 'pgsql'
# pg_port = '5432'

pg_host = '192.168.0.21'
pg_database = 'wms_legacy'
pg_user = 'postgres'
pg_password = 'iTc3D3A24#24'
pg_port = '5432'


# Connect to the FTP server
ftp = FTP(ftp_server)
ftp.login(username, password)

'230 User c011 logged in.'

specific folder and file in you want

In [8]:

# Navigate to a specific folder (replace 'target_folder' with the desired folder)
target_folder = 'Report/1 SOH'
ftp.cwd(target_folder)

# List the files in the current folder
file_list = ftp.nlst('STOCKM_C011_20240131.xlsx')
print(f"Files in '{target_folder}': {file_list}")




Files in 'Report/1 SOH': ['STOCKM_C011_20240131.xlsx']


Read Excel file directly from FTP and check the field from data

In [9]:
# Read Excel file directly from FTP
from openpyxl import load_workbook

with BytesIO() as file_buffer:
    ftp.retrbinary('RETR ' + file_list[0], file_buffer.write)
    file_buffer.seek(0)

    # Read Excel file using pandas
    df = pd.read_excel(file_buffer, engine='openpyxl')
    cek = df.columns
    print(cek)

Index(['TGL_IMPORT', '_GROUP', '_LOCATION', '_PLU_VTL', '_PLU_CUST',
       '_DESCRIPTION', '_UNIT', '_PACK', 'CF', '_F_PALLET', 'SOH', '_S_AVA',
       '_COMM', 'SOH_CAR', '_S_AVA_CAR', '_COMM_CAR', 'SOH_KG', '_S_AVA_KG',
       '_COMM_KG', '_LOT', '_TRANSC', '_REFF', '_EXP_DATE', '_WHS',
       '_CATEGORY', '_MPA_GROUP', 'BATCH', '_GROSS_WEIGHT', '_NET_WEIGHT',
       '_ROOM', '_SUB_GROUP', '_USAGE', '_SIZE', '_SUB_USAGE', '_DESC_USAGE',
       '_LOC_MAX', '_REASON_LOCK', 'EXTERNAL_ORDER_NUMBER'],
      dtype='object')


In [10]:
df.head()

,TGL_IMPORT,_GROUP,_LOCATION,_PLU_VTL,_PLU_CUST,_DESCRIPTION,_UNIT,_PACK,CF,_F_PALLET,...,_NET_WEIGHT,_ROOM,_SUB_GROUP,_USAGE,_SIZE,_SUB_USAGE,_DESC_USAGE,_LOC_MAX,_REASON_LOCK,EXTERNAL_ORDER_NUMBER
0,2024-01-31 00:42:51,CITRA DIMENSI ARTHALI,25090801,1267899,CDA-110201,TEMAN LAUT BASO UDANG 500GR (12PAK/CTN) ...,PAK,1,12,756,...,0.5,FR,N/A,Reserve,1 Pallet,N/A,Not Applicable,30240000,,IT#2401-00384
1,2024-01-31 00:42:51,CITRA DIMENSI ARTHALI,28060704,1267899,CDA-110201,TEMAN LAUT BASO UDANG 500GR (12PAK/CTN) ...,PAK,1,12,756,...,0.5,FR,N/A,Reserve,1 Pallet,N/A,Not Applicable,30240000,,IT#2401-00402
2,2024-01-31 00:42:51,CITRA DIMENSI ARTHALI,28070101,1267899,CDA-110201,TEMAN LAUT BASO UDANG 500GR (12PAK/CTN) ...,PAK,1,12,756,...,0.5,FR,N/A,Picking,1 Pallet,N/A,Not Applicable,30240000,,IT#2401-00370
3,2024-01-31 00:42:51,CITRA DIMENSI ARTHALI,28070101,1267899,CDA-110201,TEMAN LAUT BASO UDANG 500GR (12PAK/CTN) ...,PAK,1,12,756,...,0.5,FR,N/A,Picking,1 Pallet,N/A,Not Applicable,30240000,,IT#2401-00384
4,2024-01-31 00:42:51,CITRA DIMENSI ARTHALI,28080603,1267899,CDA-110201,TEMAN LAUT BASO UDANG 500GR (12PAK/CTN) ...,PAK,1,12,756,...,0.5,FR,N/A,Reserve,1 Pallet,N/A,Not Applicable,30240000,,IT#2401-00402


In [11]:
column_mapping = {
    'TGL_IMPORT': 'tgl_import',
    '_GROUP': 'group',
    '_LOCATION': 'location',
    '_PLU_VTL': 'plu_vtl',
    '_PLU_CUST': 'plu_cust',
    '_DESCRIPTION': 'description',
    '_UNIT': 'unit',
    '_PACK': 'pack',
    'CF': 'cf',
    '_F_PALLET': 'f_pallet',
    'SOH': 'soh',
    '_S_AVA': 's_ava',
    '_COMM': 'comm',
    'SOH_CAR': 'soh_car',
    '_S_AVA_CAR': 's_ava_car',
    '_COMM_CAR': 'comm_car',
    'SOH_KG': 'soh_kg',
    '_S_AVA_KG': 's_ava_kg',
    '_COMM_KG': 'comm_kg',
    '_LOT': 'lot',
    '_TRANSC': 'transc',
    '_REFF': 'reff',
    '_EXP_DATE': 'exp_date',
    '_WHS': 'whs',
    '_CATEGORY': 'category',
    '_MPA_GROUP': 'mpa_group',
    'BATCH': 'batch',
    '_GROSS_WEIGHT': 'gross_weight',
    '_NET_WEIGHT': 'net_weight',
    '_ROOM': 'room',
    '_SUB_GROUP': 'sub_group',
    '_USAGE': 'usage',
    '_SIZE': 'size',
    '_SUB_USAGE': 'sub_usage',
    '_DESC_USAGE': 'desc_usage',
    '_LOC_MAX': 'loc_max',
    '_REASON_LOCK': 'reason_lock',
    'EXTERNAL_ORDER_NUMBER': 'external_order_number'
}

# Melakukan renaming kolom berdasarkan mapping yang telah dibuat
df.rename(columns=column_mapping, inplace=True)
df.columns

Index(['tgl_import', 'group', 'location', 'plu_vtl', 'plu_cust', 'description',
       'unit', 'pack', 'cf', 'f_pallet', 'soh', 's_ava', 'comm', 'soh_car',
       's_ava_car', 'comm_car', 'soh_kg', 's_ava_kg', 'comm_kg', 'lot',
       'transc', 'reff', 'exp_date', 'whs', 'category', 'mpa_group', 'batch',
       'gross_weight', 'net_weight', 'room', 'sub_group', 'usage', 'size',
       'sub_usage', 'desc_usage', 'loc_max', 'reason_lock',
       'external_order_number'],
      dtype='object')

upload data forom ftp to postgres

In [13]:
from ftplib import FTP
from io import BytesIO
import pandas as pd
from sqlalchemy import create_engine

# Establish a connection to PostgreSQL
pg_connection_str = f"postgresql+psycopg2://{pg_user}:{pg_password}@{pg_host}:{pg_port}/{pg_database}"
engine = create_engine(pg_connection_str)

# Insert DataFrame into PostgreSQL
table_name = 'stock_ka_location_history'  # Replace with your actual table name
df.to_sql(table_name, engine, if_exists='append', index=False)

print(f"Data uploaded to PostgreSQL table '{table_name}' successfully.")

Data uploaded to PostgreSQL table 'stock_ka_location_history' successfully.


In [ ]:
# # if you have much data
# file_list = [
#     'STOCKM_C011_20240126.xlsx',
#     'STOCKM_C011_20240127.xlsx',
#     'STOCKM_C011_20240128.xlsx',
#     'STOCKM_C011_20240129.xlsx'
# ]

# filtered_files = [file for file in file_list if any(name in file for name in file_list)]

# print(f"Filtered Files in '{target_folder}': {filtered_files}")

In [16]:
# Establish a connection to PostgreSQL
connection_str = f"dbname={pg_database} user={pg_user} password={pg_password} host={pg_host} port={pg_port}"
conn = psycopg2.connect(connection_str)

# Create a cursor
cursor = conn.cursor()

# Execute SQL query to get distinct tgl_import dates
cursor.execute("SELECT DISTINCT tgl_import::date FROM stock_ka_location_history where tgl_import >= current_date")

# Fetch the results
distinct_dates = cursor.fetchall()

# Print the results
print("Distinct tgl_import dates:")
for date in distinct_dates:
    print(date[0])

Distinct tgl_import dates:
2024-01-31


Find missing dates

In [79]:
from datetime import timedelta, date

# Define the start and end dates of the range
start_date = date(2024, 1, 20)
end_date = date(2024, 1, 30)

# Generate a list of dates within the range
all_dates = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]

# Extract distinct dates from the SQL query results
distinct_dates = [date[0] for date in distinct_dates]

# Find the missing dates
missing_dates = set(all_dates) - set(distinct_dates)

# Print the missing dates
print("Missing dates:")
for missing_date in sorted(missing_dates):
        print(missing_date)

    # if missing_date == True:
    # else:        
    #     print("is nothing")


Missing dates:


In [ ]:
# Get today's date
today_date = date.today()
# day = date.current_date

# Define the SQL statement to create the view soh_master_today
create_view_sql = f"""
CREATE OR REPLACE VIEW soh_master_today AS
SELECT *
FROM soh_master
WHERE tgl_import::date = '{today_date}';
"""

# Execute the SQL statement to create the view
cursor.execute(create_view_sql)

# Define the SQL statement to create a connection between soh_master and soh_master_today
create_connection_sql = """
CREATE OR REPLACE VIEW soh_master_with_today_data AS
SELECT m.*, t.*
FROM soh_master as m
JOIN soh_master_today as t ON m.common_key = t.common_key;
"""

# Execute the SQL statement to create the connection
cursor.execute(create_connection_sql)

# Commit the changes
conn.commit()

In [56]:
ftp.quit()

'221 Goodbye!'

In [ ]:
def extract_date_from_file_name(file_list):
    match = re.search(r'_(\d{8})\.xlsx', file_list)
    if match:
        date_str = match.group(1)
        # formatted_date = pd.to_datetime(date_str, format='%Y%m%d').strftime('%Y-%m-%d')
        # return formatted_date
        return pd.to_datetime(date_str, format='%Y%m%d')
    else:
        return None


In [81]:
ftp.quit()

'221 Goodbye!'